<a href="https://colab.research.google.com/github/CassioFabius/EneSec/blob/master/Claro_Spyder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Constantes de acesso ao Claro Gestor Online:
NUMERO_DO_CLIENTE = "592573481"
USUARIO = "cassio.fabius@tvglobo.com.br"
SENHA = "@dienadmin"

# Carrega as informacoes de linhas existentes no Claro Gestor Online num DataFrame:
lista_de_linhas = DataFrame (columns=['linha'],
                             [
                                'item1',
                                'item2',
                                'item3'
                              ],
                            )

In [ ]:
# -*- coding: utf-8 -*-
# Script criado para buscar as informações do Claro Gestor Online e armazenar num arquivo JSON local.
# Criado por Cassio Fabius (cassio.fabius@g.globo)
# Atualizado em: 04/10/2020
#
# Instalar o programa "Python 3" para Windows.
# Executar os comandos abaixo no Powershell:
#   python -m pip install --upgrade pip
#   python -m pip install datetime pandas json re requests sys os time

# Bibliotecas:
import json, re, requests, sys, os, time
import datetime as dt
import pandas as pd

# Bibliotecas do serviço de envio de e-mail:
import smtplib
import email.message

# Cria um JSON (lista de dicionarios) para armazenar todas as informações:
consulta_json = []

# Inicia uma sessao para guardar cookies e se manter logado:
s = requests.Session()

# Tenta realizar o acesso ao Claro Gestor Online no maximo 3 vezes:

def tentativa_login():
        try:
            tentativa = 0
            while True:
                login_url = 'https://claro-gestoronline.claro.com.br/evpn4g/do-login'
                payload = {'enterprise': NUMERO_DO_CLIENTE, 'username': USUARIO, 'password': SENHA}
                login_response = s.post(login_url, data=payload)
        
                # Se conseguiu logar corretamente, sai do loop e segue o script:
                if login_response.status_code == 200:
                    time.sleep(2)
                    break
                else:
                    tentativa += 1
                    logger.error('Erro no login! Tentativa: ' + str(tentativa))
                    logger.error(login_response)
            
                    # Se foi a terceira tentativa, finaliza o script:
                    if tentativa > 9:
                        logger.error('Erro ao tentar carregar a página de login após 10 tentativas.')
                        sys.exit(1)

        # Se houve algum erro ao tentar realizar o login:          
        except Exception as e:
            logger.error('Erro ao tentar realizar o login.')
            logger.error(e)
            sys.exit(1)

tentativa_login()

# Conta o numero de linhas existentes:
numero_de_linhas = len(lista_de_linhas.index)

# LOOP DE LINHAS - Para cada linha da lista de linhas:
for i in range (0, numero_de_linhas, 1):
    # Armazena o numero e o nome da linha:
    numero = lista_de_linhas.iloc[(i),0]
    nome = lista_de_linhas.iloc[(i),1]
    
    # Limpa os simbolos do numero ('(62) 99106-7520' -> '62991067520'):
    numero_limpo = re.sub('\-|\(|\)|\ ', '', numero)
    login_fails = 0
    # Busca o JSON com informacoes sobre a linha:
    response = s.get('https://claro-gestoronline.claro.com.br/evpn4g/member/consumption/data/' + numero_limpo)
    #print(response)
    #print(response.text)

    #Verifica se o login foi feito com sucesso:
    if response.status_code == 401:
        login_fails = 1
        while login_fails == 1:
                tentativa_login()
                response = s.get('https://claro-gestoronline.claro.com.br/evpn4g/member/consumption/data/' + numero_limpo)
                if response.status_code == 200:
                    login_fails =0

    json_response = json.loads(response.text)

    
    # Cria um dicionario para a linha, com informacoes de numero, nome e pacotes:
    linha = {}

   

    linha['Numero'] = numero_limpo
    linha['Nome'] = nome
    #linha['Data'] = dt.datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')
    ts = time.time()
    timestamp = dt.datetime.strftime(dt.datetime.utcfromtimestamp(ts), "%Y/%m/%d %H:%M:%S"
        )
    linha['Data'] = timestamp
    
    # LOOP DE PACOTES - Para cada pacote da linha consultada:
    for item in json_response:

        # Guarda o limite do pacote (em MB, dividindo o limite duas vezes por 1024):
        limite = int(int(item['balance'][0]['initialAmount'])/1048576)
        
        # Se o limite for menor que 500GB, guarda o limite consultado:
        if limite < 500000:
            linha[str(item['offerName']) + ' - Limite (MB)'] = limite
            
        # Se o limite for maior que 500GB, o limite deve ser ilimitado, então guarda zero:
        else:
            linha[str(item['offerName']) + ' - Limite (MB)'] = 0
            
        
        # Guarda o consumo do pacote (em MB, dividindo o limite duas vezes por 1024):
        linha[str(item['offerName']) + ' - Consumo (MB)'] = int(int(item['balance'][0]['consumption'])/1048576)
        
        
        if linha[str(item['offerName']) + ' - Limite (MB)'] != 0:
            consumo_em_porcento = round((linha[str(item['offerName']) + ' - Consumo (MB)'] / linha[str(item['offerName']) + ' - Limite (MB)'])*100, 2)
            linha[str(item['offerName']) + ' - Consumo (%)'] = consumo_em_porcento
            # Para alerta de consumo no grafana:
            if consumo_em_porcento >= 80:
                linha['Alerta'] = 1
            else:
                linha['Alerta'] = 0
            
        else:
            linha[str(item['offerName']) + ' - Consumo (%)'] = 0
            linha['Alerta'] = 0


    
    # Adiciona a linha a lista ao JSON final:
    consulta_json.append(linha)

    # Substitui o formato de String para Date:

    #linha['Data'] = dt.datetime.now()
    
    #Envia a linha para o Elasticsearch:
    try:
        es.index(
            index='realtimetest', 
            ignore=400, 
            doc_type='realtime_data',
            #id= dt.datetime.now(), 
            body= json.dumps(linha),
            )
    except Exception as ex:
        logger.error("Could not send linha to Elastic Search")
        logger.error(str(ex))


    
# Exporta o JSON para o arquivo historico:
with open('C:\\Analytics\\MTM\\CLARO\\BD\\' + dt.datetime.now().strftime('%Y-%m-%d-%H') + 'h.json', 'w') as outfile:
    json.dump(consulta_json, outfile, indent=2, ensure_ascii=False)

with open('C:\\Analytics\\MTM\\CLARO\\BD\\realtime.json', 'w') as outfile:
    json.dump(consulta_json, outfile, indent=2, ensure_ascii=False)

limite_total = 0
consumo_total = 0

priority_json = []

for linha in consulta_json:
    try:
        if linha['Navegação Sem Plano com Limite - Consumo (%)'] >= 80:
            priority_json.append(linha)
        
        limite_total += linha['Navegação Sem Plano com Limite - Limite (MB)']
        consumo_total += linha['Navegação Sem Plano com Limite - Consumo (MB)']
        consumo_total += linha['Navegacao Sem Plano - Consumo (MB)']
        
    except Exception as e:
        logger.error("Linha provavelmente não possui limite configurado: " + linha['Numero'])
        logger.error(e)
        pass

total_json = []
total_json.append({
                    'Data': dt.datetime.strftime(dt.datetime.fromtimestamp(ts), "%Y/%m/%d %H:%M:%S"),
                    #'Data': dt.datetime.now(),
                    'Numero Total de Linhas': len(consulta_json),
                    'Limite Total (GB)': int(limite_total/1024),
                    'Consumo Total (GB)': int(consumo_total/1024),
                    'Consumo Total (%)': round((consumo_total/limite_total)*100, 2)
                })


#Insere total no ElasticSearch
#try:
#    es.index(
#        index='resumetest', 
#        ignore=400, 
#        doc_type='resume_data', 
#        body= json.dumps(total_json),
#            )
#except Exception as ex:
#    logger.error("Could not resume linha to Elastic Search")
#    logger.error(str(ex))

# Exporta o JSON para o arquivo de informacoes gerais:
with open('C:\\Analytics\\MTM\\CLARO\\BD\\resumo.json', 'w') as outfile:
    json.dump(total_json, outfile, indent=2, ensure_ascii=False)

with open('C:\\Analytics\\MTM\\CLARO\\Spyder\\logs\\warning.json', 'w') as outfile:
    json.dump(priority_json, outfile, indent=2, ensure_ascii=False)